In [1]:
import tensorflow as tf
import os 

In [2]:
# !pip install tqdm

  Using cached tqdm-4.64.0-py2.py3-none-any.whl (78 kB)


In [5]:
from tqdm import tqdm
from urllib import request as req

In [19]:
SOURCE_DATA_URL = "https://download.microsoft.com/download/3/E/1/3E1C3F21-ECDB-4869-8368-6DEBA77B919F/kagglecatsanddogs_5340.zip"

DESTINATION = "data"
data_file = "data.zip"
os.makedirs(DESTINATION,exist_ok=True)
DESTINATION_ZIP_PATH = os.path.join(DESTINATION,data_file)

In [20]:
file_name,headers=req.urlretrieve(SOURCE_DATA_URL,DESTINATION_ZIP_PATH)

In [21]:
file_name

'data\\data.zip'

In [23]:
print(headers)

Content-Type: application/octet-stream
Accept-Ranges: bytes
Server: Windows-Azure-Blob/1.0 Microsoft-HTTPAPI/2.0
x-ms-blob-content-md5: 4TekUHNw2UJGm20meiTqBA==
Last-Modified: Mon, 09 May 2022 20:42:07 GMT
ETag: "0x8DA31FC621E01C4"
Content-Length: 824887076
Date: Thu, 12 May 2022 14:02:34 GMT
Connection: close


